In [20]:
#Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [21]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [22]:
survey_data = pd.read_csv("./Dataset/dataset.csv")
survey_data.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Luddy or not?,other_department,luddy_department,sense of belonging _1,sense of belonging _2,sense of belonging _3,sense of belonging _4,sense of belonging _5,sense of belonging _6,sense of belonging _7,sense of belonging _8,sense of belonging _9,sense of belonging _10,sense of belonging _11,sense of belonging _12,sense of belonging _13,sense of belonging _14,sense of belonging _15,sense of belonging _16,Q19,Q12,Q13,Q15,Q16,Q17,Q14,Q6,Q10,Q10_4_TEXT,Q11,Q12.1,nationality,Q22
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,Recipient First Name,Recipient Email,External Data Reference,Location Latitude,Location Longitude,Distribution Channel,User Language,Are you a Luddy Student?,What is the name of your home department?,"If you are Luddy student, what is your home de...",Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,Please rate your agreement with the following ...,How many years have you been at IU?,I can master the content of the Luddy courses ...,I could master the content of even the most ch...,I can do a good job on all of my Luddy courses...,I can do an excellent job on Luddy course-rela...,I can learn the content taught my Luddy courses,I can earn a good grade in the Luddy courses t...,For what course are you being asked to fill ou...,How do you describe yourself? - Selected Choice,How do you describe yourself? - Prefer to self...,Choose one or more races that you consider you...,"Are you of Spanish, Hispanic, or Latino origin?",Are you a domestic or international student?,Here is your completion code for this survey. ...
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}","{""ImportId"":""recipientFirstName""}","{""ImportId"":""recipientEmail""}","{""ImportId"":""externalDataReference""}","{""ImportId"":""locationLatitude""}","{""ImportId"":""locationLongitude""}","{""ImportId"":""distributionChannel""}","{""ImportId"":""userLanguage""}","{""ImportId"":""QID8""}","{""ImportId"":""QID9_TEXT""}","{""ImportId"":""QID6""}","{""ImportId"":""QID5_1""}","{""ImportId"":""QID5_2""}","{""ImportId"":""QID5_3""}","{""ImportId"":""QID5_4""}","{""ImportId"":""QID5_5""}","{""ImportId"":""QID5_6""}","{""ImportId"":""QID5_7""}","{""ImportId"":""QID5_8""}","{""ImportId"":""QID5_9""}","{""ImportId"":""QID5_10""}","{""ImportId"":""QID5_11""}","{""ImportId"":""QID5_12""}","{""ImportId"":""QID5_13""}","{""ImportId"":""QID5_14""}","{""ImportId"":""QID5_15""}","{""ImportId"":""QID5_16""}","{""ImportId"":""QID20""}","{""ImportId"":""QID13""}","{""ImportId"":""QID14""}","{""ImportId"":""QID16""}","{""ImportId"":""QID17""}","{""ImportId"":""QID18""}","{""ImportId"":""QID15""}","{""ImportId"":""QID7_TEX

In [23]:
survey_data.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'Luddy or not?', 'other_department ', 'luddy_department ', 'sense of belonging _1', 'sense of belonging _2', 'sense of belonging _3', 'sense of belonging _4', 'sense of belonging _5', 'sense of belonging _6', 'sense of belonging _7', 'sense of belonging _8', 'sense of belonging _9', 'sense of belonging _10', 'sense of belonging _11', 'sense of belonging _12', 'sense of belonging _13', 'sense of belonging _14', 'sense of belonging _15', 'sense of belonging _16', 'Q19', 'Q12', 'Q13', 'Q15', 'Q16', 'Q17', 'Q14', 'Q6', 'Q10', 'Q10_4_TEXT', 'Q11', 'Q12.1', 'nationality', 'Q22'], dtype='object')

In [24]:
# Excluding the metadata from the analysis
survey_data = survey_data.iloc[:, 17:-1]

In [25]:
# A function to delete rows with null values
def drop_null_index(df, col):
    idx_to_drop = df.loc[df[col].isnull()].index
    df = df.drop(idx_to_drop)
    return df

In [26]:
def handle_null_values(df, col, null_count_high = False):
    # If the target variable contains null values, drop the entire row
    if col == 'sense of belonging _14':
        df = drop_null_index(df, col)
        
    # If the column contains more than 100 null values drop the column    
    if (null_count_high == True) and (col != 'sense of belonging _14'):
        df = df.drop(col, axis = 1)
        
    # If the column contains less than 100 null values, impute the column with the most common category
    if (null_count_high == False) and (col != 'sense of belonging _14'):
        fill_val = df[col].value_counts().sort_values(ascending = False).index[0]
        df[col] = df[col].fillna(fill_val)
    
    return df

In [28]:
#Mapping feature names to questions in the survey
questions_to_features_map = {key : value for key, value in zip(survey_data.iloc[0].values, survey_data.columns)}

#Drop the first two rows as they dont contain responses
survey_data = survey_data.drop([0, 1])

In [29]:
#Store the question mapping in a JSON file
with open("./Dataset/Questions_dict.json", "w") as outfile:
    json.dump(questions_to_features_map, outfile)

In [30]:
#Check null values in data
survey_data.isnull().sum()

Luddy or not?               7
other_department          194
luddy_department           54
sense of belonging _1      17
sense of belonging _2      17
sense of belonging _3      17
sense of belonging _4      17
sense of belonging _5      17
sense of belonging _6      17
sense of belonging _7      17
sense of belonging _8      17
sense of belonging _9      17
sense of belonging _10     18
sense of belonging _11     17
sense of belonging _12     17
sense of belonging _13     17
sense of belonging _14     18
sense of belonging _15     17
sense of belonging _16     17
Q19                        17
Q12                        15
Q13                        15
Q15                        15
Q16                        15
Q17                        15
Q14                        15
Q6                         17
Q10                        17
Q10_4_TEXT                239
Q11                        17
Q12.1                      17
nationality               239
dtype: int64

In [31]:
#Handling null values for different columns
survey_data = handle_null_values(survey_data, 'sense of belonging _14', null_count_high = False)
survey_data = handle_null_values(survey_data, 'other_department ', null_count_high = True)
survey_data = handle_null_values(survey_data, 'Q10_4_TEXT', null_count_high = True)
survey_data = handle_null_values(survey_data, 'nationality', null_count_high = True)
survey_data.shape

(221, 29)

In [32]:
# Reduce the long string answer to Yes/No
survey_data['Luddy or not?'] = survey_data['Luddy or not?'].apply(lambda x: str(x).split(',')[0])

In [33]:
survey_data.head()

,Luddy or not?,luddy_department,sense of belonging _1,sense of belonging _2,sense of belonging _3,sense of belonging _4,sense of belonging _5,sense of belonging _6,sense of belonging _7,sense of belonging _8,sense of belonging _9,sense of belonging _10,sense of belonging _11,sense of belonging _12,sense of belonging _13,sense of belonging _14,sense of belonging _15,sense of belonging _16,Q19,Q12,Q13,Q15,Q16,Q17,Q14,Q6,Q10,Q11,Q12.1
7,Yes,Informatics,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Yes,Yes,Yes,Yes,Yes,Yes,test survey,Non-binary / third gender,Prefer not to say,No
8,Yes,Computer Science,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,NaN,Yes,No,Yes,No,Yes,No,test survey,Prefer not to say,White or Caucasian,Yes
15,Yes,Informatics,Strongly agree,Strongly agree,Somewhat agree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Strongly agree,Somewhat disagree,Neither agree nor disagree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Neither agree nor disagree,Strongly agree,three years,Yes,No,Yes,Yes,Yes,Yes,i453,Female,White or Caucasian,No
16,Yes,Informatics,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,this is my first year,Yes,Yes,Yes,Yes,Yes,Yes,Info-453,Male,"White or Caucasian,Asian",No
17,Yes,Informatics,Somewhat agree,Strongly agree,Somewhat agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,Strongly agree,Strongly agree,Strongly agree,two years,Yes,No,Yes,Yes,Yes,Yes,Informatics Ethics - I453,Male,White or Caucasian,No


In [34]:
survey_data['luddy_department '].value_counts()

Informatics         69
Data Science        66
Computer Science    43
Other                2
Name: luddy_department , dtype: int64

In [35]:
temp = survey_data.loc[survey_data['luddy_department '].isnull() == True]
temp['Luddy or not?'].value_counts()

No     40
nan     1
Name: Luddy or not?, dtype: int64

In [36]:
survey_data.loc[survey_data['Luddy or not?'] == 'nan', 'Luddy or not?'] = 'No'
# Fill the null values in luddy_department with 'Not from luddy' as student does not belong to a Luddy department
survey_data['luddy_department '] = survey_data['luddy_department '].fillna('Not from luddy')

In [37]:
survey_data = handle_null_values(survey_data, 'sense of belonging _10', null_count_high = False)
survey_data = handle_null_values(survey_data, 'Q12.1', null_count_high = False)
survey_data = handle_null_values(survey_data, 'Q10', null_count_high = False)
survey_data = handle_null_values(survey_data, 'Q11', null_count_high = False)
survey_data = handle_null_values(survey_data, 'Q19', null_count_high = False)

In [38]:
# Storing the processed dataset to the Dataset folder
survey_data = survey_data.drop('Q6', axis = 1)
survey_data = survey_data.reset_index().drop('index', axis = 1)
survey_data.to_csv('./Dataset/Processed Data.csv', index = False)